In [1]:
import sys
import os
from scipy import stats
import itertools
import numpy as np
from colour import Color
import matplotlib.pyplot as plt
from collections import Counter
import plotly.graph_objects as go
import pandas as pd
import seaborn as sns

from matplotlib import collections  as mc

In [2]:
### bokeh imports
from bokeh.io import curdoc, output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, Select, TextInput, Slider, Button, PreText, HoverTool, Cross, Range1d, Segment, Circle, field, ImageRGBA, Rect
from bokeh.events import Tap
output_notebook()

Loading BokehJS ...

In [26]:
def read_input(file_path):
    with open(file_path, "r") as f:
        W, H, C, R = [int(a) for a in f.readline().strip().split()]
        plansza = [[{} for x in range(W)] for y in range(H)]
        c_headquarters = []
        for i in range(C):
            x, y, reward = [int(a) for a in f.readline().strip().split()]
            c_headquarters.append({"x": x, "y": y, "reward": reward})
        for i in range(H):
            line = f.readline().strip()
            for j in range(W):
                plansza[i][j]["character"] = line[j]
    return dict(
        H=H,
        W=W,
        plansza=plansza,
        C=C,
        R=R,
        c_headquarters=c_headquarters
    )

In [38]:
def read_output(file_path):
    moves = {
        "U": (0, -1),
        "D": (0, 1),
        "L": (-1, 0),
        "R": (1, 0),
    }
    paths = []
    with open(file_path, "r") as f:
        for line in f.readlines():
            x, y, directions = line.strip().split()
            x, y = int(x), int(y)
            path = [(x, y)]
            for move in directions: 
                moved = moves[move]
                path.append((path[-1][0] + moved[0], path[-1][1] + moved[1]))
            paths.append(path)
    return dict(
        paths=paths
    )

In [55]:
def get_imaging_data():
#     INPUT = read_input("2_himalayas.txt")
#     OUTPUT = read_output("2.txt")
#     INPUT = read_input("5_oceania.txt")
#     OUTPUT = read_output("5.txt")
    INPUT = read_input("4_manhattan.txt")
    OUTPUT = read_output("4.txt")
    
    print(INPUT["R"])
    
    img = np.empty((INPUT["H"],INPUT["W"]), dtype=np.uint32)
    view = img.view(dtype=np.uint8).reshape((INPUT["H"], INPUT["W"], 4))
    
    color_mapper = {
        "#": [158, 172, 176],
        "~": [52, 195, 235],
        "*": [145, 141, 16],
        "+": [191, 189, 145],
        "X": [109, 19, 148],
        "_": [55, 122, 62],
        "H": [219, 213, 33],
        "T": [199, 69, 255],
    }
    
    for i in range(INPUT["H"]):
        for j in range(INPUT["W"]):
            view[i, j] = color_mapper[INPUT["plansza"][i][j]["character"]] + [150]  # so other pixels are better visible
    
        
    for path in OUTPUT["paths"]:
        for x, y in path:
            view[y, x] = [255, 0, 0, 255]
        for x, y in path:
            view[path[0][1], path[0][0]] = [0,0,0, 255]
            
    blinkers = []
    for client_h in INPUT["c_headquarters"]:
        view[client_h["y"], client_h["x"]] = [255, 0, 0, 255]
        blinkers.append((client_h["y"], client_h["x"]))
        
        
    return dict(
        H=INPUT["H"],
        W=INPUT["W"],
        img=img,
        img_view=view,
        blinkers=blinkers,
        INPUT=INPUT,
        OUTPUT=OUTPUT,
        
    )

In [56]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import ImageRGBA, Rect
from bokeh.events import Tap

def modify_doc(doc):
    data = get_imaging_data()
    
    assert ("img" in data)
    assert ("img_view" in data)

    scale = min(950/data["W"], 2000/data["H"]) * 1.0
    p = figure(tools="pan,wheel_zoom,box_zoom,reset", 
               toolbar_location="below", 
                title=None,
                plot_width=int(data["W"] * scale),
                plot_height=int(data["H"] * scale),
                x_range=Range1d(-1, data["W"]),
                y_range=Range1d(-1, data["H"]),
    )
    p.xaxis.axis_line_width = 0
    p.yaxis.axis_line_width = 0
    p.xaxis.major_tick_in = 0
    p.yaxis.major_tick_in = 0
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None

    sources = dict()
    
    ### add image glyph
    sources["image"] = ColumnDataSource(data={"image": [data["img"]]})    
    image_glyph = ImageRGBA(image="image", x=0, y=0, dw=data["W"], dh=data["H"])
    p.add_glyph(sources["image"], image_glyph)

    ### add blinking animation for selected fields
    if "blinkers" in data:
        def animate_blinking_pixels():
            dimming_order = [255, 180, 100, 170, 255]
            for x, y in data["blinkers"]:
                try:
                    pos = dimming_order.index(data["img_view"][x, y, 3])
                    data["img_view"][x, y, 3] = dimming_order[pos + 1]
                except:
                    pass
            sources["image"].data={"image": [data["img"]]}

        doc.add_periodic_callback(animate_blinking_pixels, period_milliseconds=200)

    doc.add_root(p)

show(modify_doc)

25
